In [1]:
from keras.initializers import VarianceScaling
from keras.optimizers import Adam, Optimizer, SGD
from utils.dec import DEC
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import cv2
import numpy as np 

In [2]:
# Load CIFAR-10 dataset
(train_x, train_y), (test_x, test_y) = cifar10.load_data()
y = train_y.reshape((50000,))
y_test = test_y.reshape((10000,))

x=train_x
x_test=test_x

In [3]:
x.shape

(50000, 32, 32, 3)

In [4]:
x=x/255
x_test=x_test/255
x_test

array([[[[0.61960784, 0.43921569, 0.19215686],
         [0.62352941, 0.43529412, 0.18431373],
         [0.64705882, 0.45490196, 0.2       ],
         ...,
         [0.5372549 , 0.37254902, 0.14117647],
         [0.49411765, 0.35686275, 0.14117647],
         [0.45490196, 0.33333333, 0.12941176]],

        [[0.59607843, 0.43921569, 0.2       ],
         [0.59215686, 0.43137255, 0.15686275],
         [0.62352941, 0.44705882, 0.17647059],
         ...,
         [0.53333333, 0.37254902, 0.12156863],
         [0.49019608, 0.35686275, 0.1254902 ],
         [0.46666667, 0.34509804, 0.13333333]],

        [[0.59215686, 0.43137255, 0.18431373],
         [0.59215686, 0.42745098, 0.12941176],
         [0.61960784, 0.43529412, 0.14117647],
         ...,
         [0.54509804, 0.38431373, 0.13333333],
         [0.50980392, 0.37254902, 0.13333333],
         [0.47058824, 0.34901961, 0.12941176]],

        ...,

        [[0.26666667, 0.48627451, 0.69411765],
         [0.16470588, 0.39215686, 0.58039216]

In [5]:
import numpy as np 
x = np.reshape(x, [x.shape[0], x.shape[1] * x.shape[2]*x.shape[3]])
x_test = np.reshape(x_test, [x_test.shape[0], x_test.shape[1] * x_test.shape[2]*x_test.shape[3]])

In [6]:
x_test.shape

(10000, 3072)

CIFAR-10 with DEC

In [7]:
dims = [x.shape[-1], 500, 500, 2000, 10]

loss = 'mse'
optimizer = SGD(1., 0.9)
init = VarianceScaling(scale=1./3., mode='fan_in', distribution='uniform')
n_clusters=10

# DEC model
dec = DEC(dims=dims, n_clusters=n_clusters, initializer=init)
dec.compile(optimizer=optimizer, loss=loss)
dec.model.summary()

C:\Users\vioritiva\AppData\Roaming\Python\Python311\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 3072)]            0         
                                                                 
 encoder_0 (Dense)           (None, 500)               1536500   
                                                                 
 encoder_1 (Dense)           (None, 500)               250500    
                                                                 
 encoder_2 (Dense)           (None, 2000)              1002000   
                                                                 
 encoder_3 (Dense)           (None, 10)                20010     
                                                                 
 clustering (ClusteringLaye  (None, 10)                100       
 r)                                                              
                                                             

In [8]:
init = 'glorot_uniform'
pretrain_optimizer = 'adam'
update_interval = 140
pretrain_epochs = 1000
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                            distribution='uniform')  # [-limit, limit], limit=sqrt(1./fan_in)
pretrain_optimizer = SGD(learning_rate=1., momentum=0.9)
dec.pretrain(x=x, optimizer=pretrain_optimizer,
                    epochs=pretrain_epochs,save_dir=None)
dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')
dec.fit(x,
                update_interval=update_interval,
                save_dir=None)

Pretraining autoencoder model...
Epoch 1/1000

196/196 [==============================] - 14s 64ms/step - loss: nan
Epoch 2/1000
196/196 [==============================] - 12s 62ms/step - loss: nan
Epoch 3/1000
196/196 [==============================] - 12s 61ms/step - loss: nan
Epoch 4/1000
196/196 [==============================] - 12s 60ms/step - loss: nan
Epoch 5/1000
196/196 [==============================] - 11s 58ms/step - loss: nan
Epoch 6/1000
196/196 [==============================] - 11s 57ms/step - loss: nan
Epoch 7/1000
196/196 [==============================] - 11s 56ms/step - loss: nan
Epoch 8/1000
196/196 [==============================] - 11s 56ms/step - loss: nan
Epoch 9/1000
196/196 [==============================] - 12s 60ms/step - loss: nan
Epoch 10/1000
196/196 [==============================] - 14s 70ms/step - loss: nan
Epoch 11/1000
196/196 [==============================] - 14s 72ms/step - loss: nan
Epoch 12/1000
196/196 [==============================] - 13s 6

In [ ]:
y_pred = dec.predict(x)

In [ ]:
import numpy as np
#Creating a Mapping Between Predicted Labels and Clusters Based on Majority Labels within Each Cluster
relation = dict((x, 0) for x in range(n_clusters))
for i in range(n_clusters):
    u, indeces = np.unique(y_pred[y == i], return_inverse=True)
    i_pred = u[np.argmax(np.bincount(indeces))]
    relation[i_pred] = i
    
relation

{0: 6, 1: 0, 2: 1, 3: 8, 4: 0, 5: 0, 6: 0, 7: 0, 8: 9, 9: 7}

In [ ]:
y_pred_corr = np.array(list(map(lambda x: relation[x], y_pred)))

In [ ]:
print (y[:30])
print (y_pred_corr[:30])

[6 9 9 4 1 1 2 7 8 3 4 7 7 2 9 9 9 3 2 6 4 3 6 6 2 6 3 5 4 0]
[6 7 8 0 6 0 6 6 0 0 0 9 9 1 0 8 8 1 9 6 0 1 0 1 0 9 0 0 9 9]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_corr)

0.15272

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

normalized_mutual_info_score(y, y_pred_corr)

0.032851533750580475

In [ ]:
from sklearn.metrics import silhouette_score
# Calculate the silhouette score
silhouette_avg = silhouette_score(x, y_pred)

print("The average silhouette_score is :", silhouette_avg)

The average silhouette_score is : 0.00013756892247931727


X_test

In [ ]:
y_test.shape

(10000,)

In [ ]:
y_pred_test = dec.predict(x_test)

In [ ]:
relation = dict((x_test, 0) for x_test in range(n_clusters))
for i in range(n_clusters):
    u, indeces = np.unique(y_pred_test[y_test == i], return_inverse=True)
    i_pred = u[np.argmax(np.bincount(indeces))]
    relation[i_pred] = i
    
relation

{0: 0, 1: 0, 2: 0, 3: 0, 4: 9, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

In [ ]:
y_pred_corr_test = np.array(list(map(lambda x_test: relation[x_test], y_pred_test)))

In [ ]:
print (y_test[:30])
print (y_pred_corr_test[:30])

[3 8 8 0 6 6 1 6 3 1 0 9 5 7 9 8 5 7 8 6 7 0 4 9 5 2 4 0 9 6]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_corr_test)

0.1

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

normalized_mutual_info_score(y_test, y_pred_corr_test)

0.0

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(x_test, y_pred_corr_test)

print("The average silhouette_score is :", silhouette_avg)

Visualization

In [ ]:
#x_train
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(x)


plt.figure(figsize=(10, 8))

# Plot ground truth labels
plt.subplot(2, 1, 1)
for i in range(10):
    plt.scatter(X_tsne[y == i, 0], X_tsne[y == i, 1], label=str(i))
plt.title('Ground Truth Clustering')
plt.legend()

# Plot predicted labels
plt.subplot(2, 1, 2)
for i in range(10):
    plt.scatter(X_tsne[y_pred_corr == i, 0], X_tsne[y_pred_corr == i, 1], label=str(i))
plt.title('Predicted Clustering(x_train)')
plt.legend()

plt.show()


In [ ]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(x_test)

plt.figure(figsize=(10, 8))

# Plot ground truth labels
plt.subplot(2, 1, 1)
for i in range(10):
    plt.scatter(X_tsne[y_test == i, 0], X_tsne[y_test == i, 1], label=str(i))
plt.title('Ground Truth Clustering')
plt.legend()

# Plot predicted labels
plt.subplot(2, 1, 2)
for i in range(10):
    plt.scatter(X_tsne[y_pred_corr_test == i, 0], X_tsne[y_pred_corr_test == i, 1], label=str(i))
plt.title('Predicted Clustering(x_test)')
plt.legend()

plt.show()


In [ ]:

num_samples_to_visualize = 6
fig, axes = plt.subplots(1, num_samples_to_visualize, figsize=(10, 2))

for i in range(num_samples_to_visualize):
    # Choose a random index
    idx = np.random.randint(0, len(x))
    
    img = train_x[idx]
    y_true = train_y[idx]
    
    predicted_cluster = y_pred_corr[idx]
    
    axes[i].imshow(img)
    axes[i].set_title(f"Cluster: {predicted_cluster}\nGT Label: {y_true}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:

num_samples_to_visualize = 6
fig, axes = plt.subplots(1, num_samples_to_visualize, figsize=(10, 2))

for i in range(num_samples_to_visualize):
    # Choose a random index
    idx = np.random.randint(0, len(x_test))
    
    img = test_x[idx]
    y_true = test_y[idx]
    
    predicted_cluster = y_pred_corr_test[idx]
    
    axes[i].imshow(img)
    axes[i].set_title(f"Cluster: {predicted_cluster}\nGT Label: {y_true}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()